In [1]:
! python --version

Python 3.7.16


#Data Collection

The data that i'm gonna use is the Pandora data set, 
https://paperswithcode.com/dataset/pandora. PANDORA is the first large-scale dataset of Reddit comments labeled with different personality models (including the well-established Big 5 model and MBTI model) 

In [1]:
import pandas as pd 

In [ ]:
df=pd.read_csv("./datasets/pandora/pandora_baseline/data/author_profiles.csv")

In [ ]:
df.columns

The author profile contains differrent features suh as the author id , its  gender its age, its region , its country and the values for the personality traits.

We are interested only on the values for the MBTI and BIG5 personality traits:

'agreeableness', 'openness', 'conscientiousness', 'extraversion',
       'neuroticism',"introverted",	"intuitive",	"thinking",	"perceiving"

As in this work we aim to predict the personality of the user from its comments on the Reddit platform without taking into concideration its gender or age etc.. just from the text.


In [ ]:
df_profile=df[["author",'agreeableness', 'openness', 'conscientiousness', 'extraversion',
       'neuroticism',"introverted",	"intuitive",	"thinking",	"perceiving"]]

In [ ]:
len(df_profile.author.unique())

In [ ]:
#Verify the number of profiles that are annotqted with the Bi5 traits
df_profile[['agreeableness', 'openness', 'conscientiousness', 'extraversion','neuroticism']].dropna()

In [ ]:
#Verify the number of profiles that are annotated with the MBTI traits
df_profile[["introverted",	"intuitive",	"thinking",	"perceiving"]].dropna()

In [ ]:
#Verify the number of profiles that are annotated with both BIG5 and  MBTI traits
df_profile[['agreeableness', 'openness', 'conscientiousness', 'extraversion',
       'neuroticism',"introverted",	"intuitive",	"thinking",	"perceiving"]].dropna()

we can see that in this dataset we have exactlu 10295 unique profile where 


1.   1568 are annotated with the BIG5 traits
2.   9067 are annotated with the MBTI traits
3.   377  are annotated with both MBTI and BIG5 traits


As in this work we aim to predict both the MBTI and BIG5 traits, we will focus only on the 377 profiles that are annotated with both of them.

In [ ]:
df_profile=df_profile.dropna(subset=['introverted',	'intuitive',	'thinking',	'perceiving','agreeableness',	'openness',	'conscientiousness',	'extraversion',	'neuroticism'])
df_profile

As we can see in the previous table , the MBTI trait values {'introverted',	'intuitive',	'thinking',	'perceiving'} are binary values (they contain only 0 or 1 to define whether the profile belong to the category or not). However for the BIG5 traits, we can see that they are numerical values (for 0 to 100) which instead of indicating wether the profile belong to a category or not like the MBTI does, these values indicates to which degree a profile belong to a category. meaning that in the BIG5 traits a profile will belong to all the categories but with a degree of beloniging. Wheras in MBTI the values defines i f the user belong to the trait or not.

### Contribution
From the table above we can see that we have two problems:

1.   First we have a multi label classification task for the MBTI traits
2.   First we have a multi label regression task for the BIG5 traits



Now that we filtred the profiles that we arw gonna use, we will combine them wiith all their comments that they said since 2015.

To do that we need to call the all all_comments_since_2015.csv that comes with the Pendora package

In [ ]:
df=pd.read_csv("./datasets/pandora/all_comments_since_2015.csv")

In [ ]:
len(df)


#### all_comments_since_2015.csv

This file contain all the comments from 2015 to 2020 for all the profiles in the Pnadora dataset. It contains more than 17M comment.
However due to the fact not all profiles are annotated with both MBTI and BIG5 factors. We will keep only the comments that correspond to the filtred profiles

In [ ]:
df.columns

In [ ]:
df

In [ ]:
#we will keep only the "author",'body' and "created_utc"
df_comments=df[["author",'body',"created_utc"]]

an example of a one comment of a profile

In [ ]:

df_comments["body"][0]

#### Concatenate the filtered profiles with their comments.

In [ ]:
alld_data=pd.merge(df_comments,df_profile, on='author')

In [ ]:
len(alld_data)

In [ ]:
alld_data.dropna(inplace=True)

In [ ]:
alld_data.info()

In [ ]:
alld_data

### Feature selection for hierarchical model

In [ ]:
author_comment_counts = alld_data['body'].groupby(alld_data['author']).count()

In [ ]:
author_comment_counts

In [ ]:
authors_to_remove = author_comment_counts[author_comment_counts < 20].index

In [ ]:
authors_to_remove

In [ ]:
df_filtered = alld_data[alld_data['author'].isin(authors_to_remove)]

In [ ]:
df_filtered.dropna()

In [ ]:
filtered_indices = df_filtered.index

In [ ]:
alld_data_new = alld_data.drop(filtered_indices)


In [ ]:
alld_data_new

In [ ]:
from nltk import word_tokenize
from nltk.corpus import stopwords
import re
from string import punctuation

In [ ]:
data = alld_data_new['body'].values

In [ ]:
def preprocessing(liste):
    cleanr = re.compile(r'[http|https]*://[a-zA-Z0-9.?/&=:]*', re.S)
    raw = re.sub(cleanr, '', liste)  #去除链接
    r1 = '[{}]'.format(punctuation)
    raw = re.sub(r'\s+', ' ', raw)
    raw = re.sub(r1, '', raw)   #去除标点符号
    raw = re.sub('[\d]', '', raw)   #去除数字
    raw = raw.lower()

    token_words = word_tokenize(raw)  # 分词    

    return token_words

In [ ]:
word = []
for text in data:
    preprocessed_text = preprocessing(text)
    word.append(preprocessed_text)

In [ ]:
data_series = pd.Series(word)

In [ ]:
df = pd.DataFrame({'word': data_series})

In [ ]:
df = df.dropna()

In [ ]:
df

In [ ]:
position = 2

alld_data_procesed = pd.concat([alld_data_new.iloc[:, :position], df, alld_data_new.iloc[:, position:]], axis=1)

In [ ]:
alld_data_procesed = alld_data_procesed.dropna(subset=['word'])

In [ ]:

alld_data_procesed['word'] = alld_data_procesed['word'].apply(lambda x: ' '.join(x))


In [ ]:
alld_data_procesed

In [ ]:
import random

In [ ]:
import pandas as pd

target_word_count = 70  # 每个句子的目标单词数
target_doc_size = 40   # 每个用户的目标评论数

# 按作者将评论分组
grouped_data = alld_data_procesed.groupby('author')['word'].apply(list)

# 初始化一个新的DataFrame来存储修改后的数据
new_data = pd.DataFrame(columns=['author', 'word'])

for user_id, comments in grouped_data.items():
    user_chunks = []  # 用于存储每40个评论拼接的文本块
    chunk = []  # 用于存储当前评论的单词列表

    for comment in comments:
        comment_words = comment.split()

        # 截取每个句子的前40个单词，不足的部分用[PAD]填充
        for word in comment_words[:target_word_count]:
            chunk.append(word)

        # 如果不足40个单词，用[PAD]填充
        while len(chunk) < target_word_count:
            chunk.append('[PAD]')

        # 将当前句子的单词列表添加到文本块中
        user_chunks.append(chunk)
        chunk = []

        # 如果达到40个评论，拼接为一个文本块
        if len(user_chunks) == target_doc_size:
            new_data = new_data.append({'author': user_id, 'word': user_chunks}, ignore_index=True)
            user_chunks = []

    # 检查用户评论数量是否多于20个
    if len(user_chunks) > 20 or len(user_chunks) == 20:
        # 计算需要填充的评论数量
        pad_count = target_doc_size - len(user_chunks)
        # 填充到40个评论
        for _ in range(pad_count):
            user_chunks.append(['[PAD]'] * target_word_count)

    # 检查用户评论数量是否小于20个，如果小于20个，则跳过该用户的数据
    if len(user_chunks) < 20:
        continue

    # 创建一个包含该用户数据的DataFrame，并将其附加到new_data DataFrame中
    new_data = new_data.append({'author': user_id, 'word': user_chunks}, ignore_index=True)


In [ ]:
new_data

In [ ]:
alld_data_procesed

In [ ]:
other_columns = ['agreeableness','openness','conscientiousness','extraversion','neuroticism','introverted','intuitive','thinking','perceiving']

In [ ]:
author_data = alld_data_procesed.groupby('author')[other_columns].first().reset_index()

# 将提取的数据添加到df2中，并保持与作者的对应关系
merged_df = pd.merge(new_data, author_data, on='author', how='left')

In [ ]:
merged_df

In [ ]:
merged_df.to_csv('datasets/pandora/data_final.csv', index=False)

# Model creation

Now that we filtred and created the data that we need, we will start now by creating the learning model architecture.

To do that in a nutshell we will use the hugging face librarie to fune-tune a pretrained model so that it encodes the comments of the profile in a contextual way. Then using thease vector embeddings we will train a multi-task neural network to predict both MBTI and BIG5 values. All of this is done in an END-To-END network.

We will start firs by importing all the necessary libraries

In [ ]:

 
!pip install transformers


In [ ]:
!pip install audtorch

In [ ]:
!nvidia-smi


In [3]:
import os
import math
import audtorch
import torch
from torch.nn import BCEWithLogitsLoss,MSELoss
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, BertTokenizer, BertModel, BertConfig

from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from tqdm import tqdm, trange
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
!pip install transformers==3.3.1

In [ ]:
!pip install matplotlib

In [ ]:
!pip uninstall pillow


Before start creating the model we need first to prepare our data so that thei'll bee appropriate to be consumed by our model. 


We need first to devide our data into training, test and validation sets.


1.   The train data will be used as the main data to train the model
2.   the validation data will be used also during the training phase but just to validate the performance of the model during the training phase.
3.   The test data will be used as a holdout data that has not been seen by the model in order to truly evaluate the generalization of our model

We will use 16726 of dqtq rows to train the model, 4182 as a validation data and 5227 as test data


In [15]:
merged_df = pd.read_csv('./datasets/pandora/data_final.csv')
merged_df['word'] = merged_df['word'].apply(eval).tolist()

In [16]:
from sklearn.model_selection import train_test_split
train,test = train_test_split(merged_df, test_size=0.2,random_state=0)
train,valid = train_test_split(train, test_size=0.2,random_state=0)

In [17]:
len(train)

16726

In [18]:
len(valid)

4182

In [19]:
len(test)

5227

In [20]:
model_name='bert-base-uncased'

tokenizer = BertTokenizer.from_pretrained(model_name, do_lower_case=True)

train_data= train["word"].values
test_data = test["word"].values
valid_data = valid["word"].values



Now, we extract emotion features

In [ ]:
import senticnet7

In [ ]:
import pandas as pd
lexion_df = pd.read_excel('/home/tjutuser3/tjut_han/emotion /NRC-Sentiment-Emotion-Lexicons/NRC-Emotion-Lexicon-v0.92/NRC-Emotion-Lexicon-v0.92-In105Languages-Nov2017Translations.xlsx')

lexion_df.columns.tolist()

english_df = lexion_df[['English (en)','Positive', 'Negative','Anger','Anticipation', 'Disgust', 'Fear','Joy', 'Sadness', 'Surprise', 'Trust']]
english_df.head()


In [37]:
with open('/home/tjutuser3/tjut_han/emotion /DepecheMood_english_token_full.tsv', 'r', encoding='utf-8') as file:
    emotion_dict_token = {}  

   
    skip_first_line = True

    for line in file:
        if skip_first_line:
            skip_first_line = False
            continue  

        parts = line.strip().split('\t')

        if len(parts) >= 1:
            word = parts[0] 
            emotion_info = parts[1:11] 
            emotion_dict_token[word] = emotion_info


In [38]:
with open('/home/tjutuser3/tjut_han/emotion /DepecheMood_english_lemma_full.tsv', 'r', encoding='utf-8') as file:
    emotion_dict_lemma = {}  
    skip_first_line = True

    for line in file:
        if skip_first_line:
            skip_first_line = False
            continue 
        parts = line.strip().split('\t')
        if len(parts) >= 1:
            word = parts[0]
            emotion_dict_lemma[word] = emotion_info


In [39]:
emotion_dict_lemma

{'aa': ['0.0',
  '0.6425323778032789',
  '0.0',
  '0.25961468748894084',
  '0.09785293470778024',
  '0.0',
  '0.0',
  '0.0',
  '1'],
 'aaa': ['0.0',
  '0.6425323778032789',
  '0.0',
  '0.25961468748894084',
  '0.09785293470778024',
  '0.0',
  '0.0',
  '0.0',
  '1'],
 'aaaaah': ['0.0',
  '0.6425323778032789',
  '0.0',
  '0.25961468748894084',
  '0.09785293470778024',
  '0.0',
  '0.0',
  '0.0',
  '1'],
 'aaaaahhhh': ['0.0',
  '0.6425323778032789',
  '0.0',
  '0.25961468748894084',
  '0.09785293470778024',
  '0.0',
  '0.0',
  '0.0',
  '1'],
 'aaaaand': ['0.0',
  '0.6425323778032789',
  '0.0',
  '0.25961468748894084',
  '0.09785293470778024',
  '0.0',
  '0.0',
  '0.0',
  '1'],
 'aaaahhs': ['0.0',
  '0.6425323778032789',
  '0.0',
  '0.25961468748894084',
  '0.09785293470778024',
  '0.0',
  '0.0',
  '0.0',
  '1'],
 'aaaahs': ['0.0',
  '0.6425323778032789',
  '0.0',
  '0.25961468748894084',
  '0.09785293470778024',
  '0.0',
  '0.0',
  '0.0',
  '1'],
 'aaaand': ['0.0',
  '0.6425323778032789',


In [40]:
emotion_dict_token

{'aa': ['0.0054066991014751825',
  '0.010635989194095686',
  '0.011927524123865642',
  '0.010163990269009415',
  '0.19470465335113124',
  '0.08262850924065102',
  '0.6797990668088009',
  '0.004733567910970831',
  '45'],
 'aaa': ['0.16796334886111175',
  '0.10240247656442168',
  '0.06830559491317141',
  '0.06058203929008373',
  '0.08176409651532653',
  '0.28094407432467905',
  '0.20578001407218632',
  '0.03225835545901964',
  '59'],
 'aaaaah': ['0.0035476267414188425',
  '0.03259930020675789',
  '0.0475257414260463',
  '0.047537475239616545',
  '0.1674683694780379',
  '0.1788115913246051',
  '0.4621911299343551',
  '0.06031876564916225',
  '3'],
 'aaaaahhhh': ['0.014938542070713004',
  '0.28084835810275444',
  '0.018741195808847675',
  '0.20017341889763607',
  '0.03772428416720053',
  '0.3861779845218377',
  '0.047808681883108044',
  '0.01358753454790268',
  '2'],
 'aaaaand': ['0.0',
  '0.05202530615563045',
  '0.01424505304580042',
  '0.684676927682728',
  '0.07168491735502312',
  '0.0

In [41]:
with open('/home/tjutuser3/tjut_han/emotion /NRC-VAD-Lexicon.txt', 'r', encoding='utf-8') as file:
    NRC_VAD_dict = {}  
    for line in file:
        parts = line.strip().split('\t')
        if len(parts) >= 1:
            word = parts[0]  
            emotion_info = parts[1:4]  
            NRC_VAD_dict[word] = emotion_info
NRC_VAD_dict


{'aaaaaaah': ['0.479', '0.606', '0.291'],
 'aaaah': ['0.520', '0.636', '0.282'],
 'aardvark': ['0.427', '0.490', '0.437'],
 'aback': ['0.385', '0.407', '0.288'],
 'abacus': ['0.510', '0.276', '0.485'],
 'abalone': ['0.500', '0.480', '0.412'],
 'abandon': ['0.052', '0.519', '0.245'],
 'abandoned': ['0.046', '0.481', '0.130'],
 'abandonment': ['0.128', '0.430', '0.202'],
 'abashed': ['0.177', '0.644', '0.307'],
 'abate': ['0.255', '0.696', '0.604'],
 'abatement': ['0.388', '0.338', '0.336'],
 'abba': ['0.562', '0.500', '0.480'],
 'abbey': ['0.580', '0.367', '0.444'],
 'abbot': ['0.427', '0.321', '0.483'],
 'abbreviate': ['0.531', '0.375', '0.330'],
 'abbreviation': ['0.469', '0.306', '0.345'],
 'abdomen': ['0.469', '0.462', '0.471'],
 'abdominal': ['0.490', '0.456', '0.445'],
 'abduct': ['0.173', '0.720', '0.615'],
 'abduction': ['0.062', '0.990', '0.673'],
 'aberrant': ['0.146', '0.765', '0.431'],
 'aberration': ['0.125', '0.816', '0.417'],
 'abeyance': ['0.330', '0.510', '0.292'],
 'ab

In [42]:
Positive, Negative, Anger, Anticipation, Disgust, Fear, Joy, Sadness, Surprise, Trust= [[] for i in range(10)]
for idx, row in english_df.iterrows():
    if row['Positive']==1:
        Positive.append(row['English (en)'])
    if row['Negative']==1:
        Negative.append(row['English (en)'])
    if row['Anger']==1:
        Anger.append(row['English (en)'])
    if row['Anticipation']==1:
        Anticipation.append(row['English (en)'])
    if row['Disgust']==1:
        Disgust.append(row['English (en)'])
    if row['Fear']==1:
        Fear.append(row['English (en)'])
    if row['Joy']==1:
        Joy.append(row['English (en)'])
    if row['Sadness']==1:
        Sadness.append(row['English (en)'])
    if row['Surprise']==1:
        Surprise.append(row['English (en)'])
    if row['Trust']==1:
        Trust.append(row['English (en)'])
 
print('词语列表构建完成')

词语列表构建完成


In [43]:
from nltk.corpus import sentiwordnet as swn
from nltk.sentiment.vader import SentimentIntensityAnalyzer


In [23]:
class SenticNet(object):
    """
    Simple API to use SenticNet.
    """
    def __init__(self):
        self.data = senticnet7.senticnet

    # public methods

    def concept(self, concept):
        """
        Return all the information about a concept: polarity, sentics, moodtags, and semantics.
        """
        result = {}

        result["polarity_label"] = self.polarity_label(concept)
        result["polarity_value"] = self.polarity_value(concept)
        result["sentics"] = self.sentics(concept)
        result["moodtags"] = self.moodtags(concept)
        result["semantics"] = self.semantics(concept)

        #introspection_value', 'temper_value', 'attitude_value', 'sensitivity_value', 'primary_emotion', 'secondary_emotion', 'polarity_label', 'polarity_value',

        return result

    def sentics(self, concept):
        """
        Return the sentics of a concept.
        """
        # print('Return the sentics of a concept')
        concept = concept.replace(" ", "_")
        # concept_info = self.data.get(concept)
        concept_info = self.data[concept]
        # print(concept_info[0])

        sentics = {"introspection": concept_info[0],
                   "temper": concept_info[1],
                   "attitude": concept_info[2],
                   "sensitivity": concept_info[3]}

        return sentics

    def moodtags(self, concept):
        """
        Return the moodtags of a concept.
        """
        concept = concept.replace(" ", "_")
        concept_info = self.data[concept]

        return concept_info[4:6]

    def polarity_label(self, concept):
        """
        Return the polarity label of a concept.
        """
        concept = concept.replace(" ", "_")
        concept_info = self.data[concept]

        return concept_info[6]

    def polarity_value(self, concept):
        """
        Return the polarity value of a concept.
        """
        concept = concept.replace(" ", "_")
        concept_info = self.data[concept]

        return concept_info[7]

    def semantics(self, concept):
        """
        Return the semantics associated with a concept.
        """
        concept = concept.replace(" ", "_")
        concept_info = self.data[concept]

        return concept_info[8:]

In [32]:
import numpy as np
from senticnet.senticnet import SenticNet
import torch

def get_sent_score(input_list):
    sn = SenticNet()
    sia = SentimentIntensityAnalyzer()

    emotion_vectors = []
    positive, negative, anger, anticipation, disgust, fear, joy, sadness, surprise, trust = [0 for i in range(10)]

    for wordlist in input_list:
        word_num = 0
        emotion_score = 0
        introspection =0
        temper = 0
        attitude = 0
        sensitivity = 0
        AFRAID_token = 0
        AMUSED_token= 0
        ANGRY_token	= 0
        ANNOYED_token = 0
        DONT_CARE_token = 0
        HAPPY_token = 0
        INSPIRED_token = 0
        SAD_token = 0
        AFRAID_lemma = 0
        AMUSED_lemma = 0
        ANGRY_lemma	= 0
        ANNOYED_lemma = 0
        DONT_CARE_lemma = 0
        HAPPY_lemma = 0
        INSPIRED_lemma = 0
        SAD_lemma = 0
        Valence	= 0
        Arousal	= 0
        Dominance = 0
        
        sentence = " ".join(wordlist)
        sentiment_scores = sia.polarity_scores(sentence)

        for word in wordlist:
            try:
                concept_info = sn.concept(word)
                polarity_value = sn.polarity_label(word)
                polarity_intense = sn.polarity_value(word)
                
                sentics = sn.sentics(word)
                introspection_value = sentics['introspection']
                temper_value = sentics['temper']
                attitude_value = sentics['attitude']
                sensitivity_value = sentics['sensitivity']

                introspection += float(introspection_value)
                temper += float(temper_value)
                attitude += float(attitude_value)
                sensitivity += float(sensitivity_value)
                
                emotion_score += float(polarity_intense)
                word_num += 1
                
                freq = wordlist.count(word)
                if word in Positive:
                    positive+=freq
                if word in Negative:
                    negative+=freq
                if word in Anger:
                    anger+=freq  
                if word in Anticipation:
                    anticipation+=freq
                if word in Disgust:
                    disgust+=freq
                if word in Fear:
                    fear+=freq
                if word in Joy:
                    joy+=freq
                if word in Sadness:
                    sadness+=freq
                if word in Surprise:
                    surprise+=freq
                if word in Trust:
                    trust+=freq
                if word in emotion_dict_token:
                    emotion_info_token = emotion_dict_token[word]
                    
                    AFRAID_token += float(emotion_info_token[0])
                    AMUSED_token += float(emotion_info_token[1])
                    ANGRY_token	+= float(emotion_info_token[2])
                    ANNOYED_token += float(emotion_info_token[3])
                    DONT_CARE_token += float(emotion_info_token[4])
                    HAPPY_token += float(emotion_info_token[5])
                    INSPIRED_token += float(emotion_info_token[6])
                    SAD_token += float(emotion_info_token[7])

                if word in emotion_dict_lemma:
                    emotion_info_lemma = emotion_dict_lemma[word]
                    
                    AFRAID_lemma += float(emotion_info_lemma[0])
                    AMUSED_lemma	+= float(emotion_info_lemma[1])
                    ANGRY_lemma	+= float(emotion_info_lemma[2])
                    ANNOYED_lemma	+= float(emotion_info_lemma[3])
                    DONT_CARE_lemma += float(emotion_info_lemma[4])
                    HAPPY_lemma += float(emotion_info_lemma[5])
                    INSPIRED_lemma +=	float(emotion_info_lemma[6])
                    SAD_lemma += float(emotion_info_lemma[7])
                if word in NRC_VAD_dict:
                    emotion_info_NRC = NRC_VAD_dict[word]
                    
                    Valence	+= float(emotion_info_NRC[0])
                    Arousal	+= float(emotion_info_NRC[1])
                    Dominance += float(emotion_info_NRC[2])
                   
                
            except:
                continue
                
                
        if word_num > 0:
            polarity_value = emotion_score 
            introspection_value = introspection 
            temper_value = temper 
            attitude_value = attitude 
            sensitivity_value = sensitivity 
        else:
            polarity_value = 0
            introspection_value = 0
            temper_value = 0
            attitude_value = 0
            sensitivity_value = 0

        emotion_vector = [polarity_value ,introspection_value,temper_value,attitude_value,sensitivity_value,
                          positive, negative, anger, anticipation, disgust, fear, joy, sadness, surprise, trust,
                          sentiment_scores['neg'],sentiment_scores['neu'],sentiment_scores['pos'],sentiment_scores['compound'],
                          AFRAID_token,AMUSED_token,ANGRY_token,ANNOYED_token,DONT_CARE_token,HAPPY_token,INSPIRED_token,SAD_token,
                          AFRAID_lemma,AMUSED_lemma,ANGRY_lemma,ANNOYED_lemma,DONT_CARE_lemma,HAPPY_lemma,INSPIRED_lemma,SAD_lemma,
                          Valence,Arousal,Dominance]
       
        emotion_vectors.append(emotion_vector)

    return emotion_vectors


In [ ]:
train_data_emotion_vector = []
for text in train_data:
#     text = text.split()
    preprocessed_text = get_sent_score(text)
    train_data_emotion_vector.append(preprocessed_text)
train_data_emotion_vector = torch.tensor(train_data_emotion_vector, dtype=torch.float64)

In [ ]:
train_data_emotion_vector.shape

In [ ]:
test_data_emotion_vector = []
for text in test_data:
#     text = text.split()
    preprocessed_text = get_sent_score(text)
    test_data_emotion_vector.append(preprocessed_text)
test_data_emotion_vector = torch.tensor(test_data_emotion_vector, dtype=torch.float64)

In [ ]:
valid_data_emotion_vector = []
for text in valid_data:
#     text = text.split()
    preprocessed_text = get_sent_score(text)
    valid_data_emotion_vector.append(preprocessed_text)
valid_data_emotion_vector = torch.tensor(valid_data_emotion_vector, dtype=torch.float64)

If we take a closer look to the train_input_ids wich represent the encoding of each row in the training data we can see that the row of texts were transformed into rows of vector of numbers, where each vector of numbers represent a textual comment and each number in the vector represent a word identificator in the comment. The numbers are token representation were each number refer to a word in the pre_trained BERT vocabulary

In [ ]:
def Input_Tokenization(dataset, tokenizer, max_user_comments=40, max_comment_length=70):
    input_ids_list = []
    attention_mask_list = []

    for user_comments in dataset:
        user_input_ids = []
        user_attention_mask = []

        for comment in user_comments:
            comment_text = ' '.join(comment)
            encoded_comment = tokenizer.encode_plus(
                comment_text,
                padding='max_length',
                truncation=True,
                max_length=max_comment_length,
                return_tensors='pt',
                return_attention_mask=True
            )

            input_ids = encoded_comment['input_ids']
            attention_mask = encoded_comment['attention_mask']

            user_input_ids.append(input_ids)
            user_attention_mask.append(attention_mask)

        input_ids_list.append(user_input_ids)
        attention_mask_list.append(user_attention_mask)

    input_ids_tensor = torch.stack([torch.cat(user_input_ids) for user_input_ids in input_ids_list])
    attention_mask_tensor = torch.stack([torch.cat(user_attention_mask) for user_attention_mask in attention_mask_list])

    return input_ids_tensor, attention_mask_tensor

In [ ]:
train_input_ids, train_attention_masks = Input_Tokenization(train_data, tokenizer)

In [ ]:
train_input_ids

In [ ]:
test_input_ids, test_attention_masks = Input_Tokenization(test_data, tokenizer)


In [ ]:
valid_input_ids, valid_attention_masks = Input_Tokenization(valid_data, tokenizer)

Now that we transformed our data into a numerical values 'vectors of numbers' instead of the original textual values we will now transform them into a convininet form to be consumed by the Pytorch platform

To do that we need first to define the names of label features

In [ ]:
label_cols = ['agreeableness', 'openness',
       'conscientiousness', 'extraversion', 'neuroticism', 'introverted',
       'intuitive', 'thinking', 'perceiving']

Like we said, our model is a multi task prediction model. so we have classification labels and regression labels.

In [ ]:
#Classification labels
clas_label_cols=[ 'introverted',
       'intuitive', 'thinking', 'perceiving']

In [ ]:
#Regression labels
reg_label_cols=['agreeableness', 'openness',
       'conscientiousness', 'extraversion', 'neuroticism']

In [ ]:


X_train,train_masks,Y_train_r,Y_train_c = (train_input_ids,train_attention_masks,torch.tensor(train[reg_label_cols].values.tolist(),dtype=torch.float32),torch.tensor(train[clas_label_cols].values.tolist(),dtype=torch.float32))
X_valid,valid_masks,Y_valid_r,Y_valid_c = (valid_input_ids,valid_attention_masks,torch.tensor(valid[reg_label_cols].values.tolist(),dtype=torch.float32),torch.tensor(valid[clas_label_cols].values.tolist(),dtype=torch.float32))


During the training phase we will need for both the training set and the validation set. Meaning that we will need for both the training and validation vector representations as well as the the classification and regression labels.

For each row we need to  have the tokenization(ids and mask) of that row and ist MBTI(classification labels) and BIG5(regression labels) values.

if we take a closer look to the first row information in the training data we can find :

In [ ]:
X_valid# the comment representation 'tokenization' of the first row
#We can see a lot of 0 values. This is because the vector size of the pre-trqined Electra model id 256. and the nuber of words in the first comment are less than 256 
#So the 0 values are added as a padding values to make all the vectors have the same size

In [ ]:
valid_masks# The mask represent binary values 0 or 1
#1 means that the nodel will take into concideration the word in that position while 0 means that the model
#will not take into concideration that word.
#This is logic: as we can see all the 0 are puted in the position of the padding values in the above 'x_train[0]' comments.
#which mean that the model will not concentrate on the padding values. It will concentrate only on the comment words values

In [ ]:
Y_train_r
# The classification labels 'MBTI labels' for the first comment are 

In [ ]:
len(Y_trains_r[0])
# The regression labels 'Big5 labels' for the first comment are 

Now that we know how our data looks like and the meaning of each data representation we will transform the data in a Pytorch format usign the data loader function to make them in chunks in order to trqin our model in a batch way

In [ ]:
batch_size = 140



train_data = TensorDataset(X_train, train_masks, Y_train_r,Y_train_c,train_data_emotion_vector)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data,\
                              sampler=train_sampler,\
                              batch_size=batch_size
                             )

validation_data = TensorDataset(X_valid, valid_masks, Y_valid_r,Y_valid_c,valid_data_emotion_vector)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data,\
                                   sampler=validation_sampler,\
                                   batch_size=batch_size
                                  )

In [ ]:
import torch
from torch import nn
from typing import Tuple
class Attention(nn.Module):
    def __init__(self, input_size: int) -> None:
        super(Attention, self).__init__()
        self.w = nn.Linear(input_size, 1)
        self.tanh = nn.Tanh()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, H: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        M = self.tanh(H)  
        alpha = self.w(M).squeeze(2)  
        alpha = self.softmax(alpha)  
        r = H * alpha.unsqueeze(2) 
        r = r.sum(dim = 1) 

        return r, alpha

In [ ]:
class TextCNN(nn.Module):
    def __init__(self, input_channels, output_channels, kernel_sizes, input_length):
        super(TextCNN, self).__init__()
        self.input_channels = input_channels
        self.output_channels = output_channels
        self.kernel_sizes = kernel_sizes
        self.convs = nn.ModuleList([
            nn.Conv1d(input_channels, output_channels, kernel_size) for kernel_size in kernel_sizes
        ])
        self.input_length = input_length

    def forward(self, x):
        x = x.permute(0, 2, 1)
        x = [conv(x) for conv in self.convs]
        x = [torch.max_pool1d(conv, self.input_length - kernel_size + 1) for conv, kernel_size in zip(x, self.kernel_sizes)]
        x = torch.cat(x, dim=-1)
        x = x.permute(0, 2, 1)


        return x


In [ ]:
from torch.nn import BCELoss

Now that the data is ready we will create the hierarchical attention network

In [ ]:
class BertSharedWeights_Multi_Task(torch.nn.Module):

    def __init__(self, r_num_labels=2, c_num_labels=2, input_size=None, hidden_size=None, num_layers=None):
        super(BertSharedWeights_Multi_Task, self).__init__()
        self.c_num_labels = c_num_labels
        self.r_num_labels = r_num_labels
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.bert = BertModel.from_pretrained(model_name,output_hidden_states=True)
        self.attention = Attention(input_size=512*2)
        self.CNN_attention = Attention(input_size=512)
        self.drop = torch.nn.Dropout(p=0.5)
        self.bigru = torch.nn.GRU(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True, bidirectional=True)
        self.layer1 = torch.nn.Linear(512*3,512*2)# add The first linear layer on top of the encoding  discriminator layers of Bert
        self.layer2  = torch.nn.Linear(512*2, 512)# add the second linear layer on top of the first one
        self.layer3  = torch.nn.Linear(512, 256)# add the third linear layer on top of the second one
        self.classifier = torch.nn.Linear(256, c_num_labels)
        self.regressor = torch.nn.Linear(256, r_num_labels)
        self.leaky = torch.nn.LeakyReLU()
        self.tanh = torch.nn.Tanh()

        self.text_cnn = TextCNN(input_channels=input_size, output_channels=hidden_size, kernel_sizes=[3,4,5],input_length=40)
        torch.nn.init.xavier_normal_(self.layer1.weight) #initialise the weight of the first linear layer
        torch.nn.init.xavier_normal_(self.layer2.weight) #initialise the weight of the second linear layer
        torch.nn.init.xavier_normal_(self.layer3.weight) #initialise the weight of the third linear layer
        torch.nn.init.xavier_normal_(self.classifier.weight)
        torch.nn.init.xavier_normal_(self.regressor.weight)
        self.l1_lambda = 0.00078  

    def forward(self, input_ids_list, emotion_vectors, token_type_ids=None,
                attention_mask=None, labels_r=None, labels_c=None):
        user_contextual_embeddings = []
 
        for i in range(len(input_ids_list)):
            Bert_hidding_state = self.bert(input_ids=input_ids_list[i],
                                           attention_mask=attention_mask[i],
                                           token_type_ids=token_type_ids)
        #     print(Bert_hidding_state.shape)
            Bert_hidding_states = Bert_hidding_state[2]
            contextual_embedding = Bert_hidding_states[11][: ,0 ,: ]
            contextual_embedding = torch.cat((contextual_embedding, emotion_vectors[i]), dim=1)
#             print(contextual_embedding.shape)
            user_contextual_embeddings.append(contextual_embedding)
#         print(len(user_contextual_embeddings))
        contextual_embeddings = torch.stack(user_contextual_embeddings, dim=0)
        contextual_embeddings = contextual_embeddings.to(torch.float)
#         print(contextual_embeddings.shape)
        sentence1, _ = self.bigru(contextual_embeddings)
        sentence_embedding,alphas = self.attention(sentence1)
        sentence_embeddings1 = self.tanh(sentence_embedding)
        sentence2 = self.text_cnn(contextual_embeddings)
        # print(sentence2.shape)
        sentence_embedding1,alphas1 = self.CNN_attention(sentence2)
        sentence_embeddings2 = self.tanh(sentence_embedding1)
        # print(sentence_embeddings1.shape,sentence_embeddings2.shape)
        sentence_embeddings = torch.cat((sentence_embeddings1,sentence_embeddings2),dim=1)
#         print(sentence_embedding.shape)

        cl1 = self.leaky(self.layer1(sentence_embeddings))
        cl1=self.drop(cl1)
        cl2 = self.leaky(self.layer2(cl1))
        cl2=self.drop(cl2)
        cl3 = self.leaky(self.layer3(cl2))
        cl3=self.drop(cl3)
        
        clogits = self.classifier(cl3)

        rlogits = self.regressor(cl3)
        
        closs_fct = BCEWithLogitsLoss()
        rloss_fct = MSELoss()

        if labels_r is not None and labels_c is not None:
            closs = closs_fct(clogits.view(-1, self.c_num_labels),
                              labels_c.view(-1, self.c_num_labels))
            rloss = rloss_fct(rlogits.view(-1, self.r_num_labels),
                              labels_r.view(-1, self.r_num_labels))

            loss = (0.6*rloss + 0.4*closs) / 2
            
            l1_norm = sum(p.abs().sum() for p in self.parameters()) 
            loss += self.l1_lambda * l1_norm 

            return loss
        else:
            return clogits, rlogits


In [ ]:
len(Y_train_c)

Noow that we created the architecture of our model we will instanciate it, make it capable of runing in paralell on defining it to be running on GPUs

In [ ]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = "1,2,3,4,5,6,7"
device_ids = [0,1,2,3,4,5,6]
device = "cuda"
model = BertSharedWeights_Multi_Task(c_num_labels=len(Y_train_c[0]),r_num_labels=len(Y_train_r[0]),input_size=806,hidden_size=512,num_layers=2)
model = torch.nn.DataParallel(model,device_ids = device_ids)
if torch.cuda.is_available():
    model.cuda()
torch.cuda.empty_cache()

In [ ]:
!pip install hiddenlayer 
!pip install torchviz 


In [ ]:

import torchvision.models
import hiddenlayer as hl

In [ ]:
'''x = np.zeros((15, 256))
x=torch.LongTensor(x)
hl.build_graph(model, x.to('cuda'))'''

We also need to define the Optimizer that we will use during the learning process for the gradient propagation. We will use the popular Adam optimizer with a 0.01 weight decay and a 4e-5 learning rate

In [ ]:
optimizer = AdamW(model.parameters(), lr=4e-5, weight_decay=0.01, correct_bias=True)

Now we will define the training function for our model

In [ ]:
def train(model, num_epochs,\
          optimizer,\
          train_dataloader, valid_dataloader,\
          model_save_path,\
          train_loss_set=[], valid_loss_set = [],\
          lowest_eval_loss=None, start_epoch=0,\
          device="cpu"
          ):
  """
  Train the model and save the model with the lowest validation loss
  """

  model.to(device)

  for i in trange(num_epochs, desc="Epoch"):
    
    actual_epoch = start_epoch + i

    

    model.train()

    tr_loss = 0
    num_train_samples = 0

    for step, batch in enumerate(train_dataloader):
      # Add batch to GPU
      batch = tuple(t.to(device) for t in batch)
      # Unpack the inputs from our dataloader
      b_input_ids, b_input_mask, b_labels_r,b_labels_c,emotion_vectors = batch
      b_input_ids = b_input_ids.to(device)
      b_input_mask = b_input_mask.to(device)
      b_labels_r = b_labels_r.to(device)
      b_labels_c = b_labels_c.to(device)
      emotion_vectors = emotion_vectors.to(device)
      # Clear out the gradients (by default they accumulate)
      optimizer.zero_grad()
      # Forward pass
      loss = model(b_input_ids,emotion_vectors, attention_mask=b_input_mask, labels_r=b_labels_r,labels_c=b_labels_c)
      # store train loss
      loss = loss.mean()
      tr_loss += loss.item()
#       tr_loss +=loss.tolist()
      num_train_samples += b_labels_c.size(0)
      # Backward pass
      loss.backward()
      # Update parameters and take a step using the computed gradient
      optimizer.step()
      #scheduler.step()

    # Update tracking variables
    epoch_train_loss = tr_loss/num_train_samples
    train_loss_set.append(epoch_train_loss)

    print("Train loss: {}".format(epoch_train_loss))

    # Validation

    # Put model in evaluation mode to evaluate loss on the validation set
    model.eval()

    # Tracking variables 
    eval_loss = 0
    num_eval_samples = 0

    # Evaluate data for one epoch
    
    for batch in valid_dataloader:
      # Add batch to GPU
      batch = tuple(t.to(device) for t in batch)
      # Unpack the inputs from our dataloader
      b_input_ids, b_input_mask, b_labels_r,b_labels_c,emotion_vectors = batch
      b_input_ids = b_input_ids.to(device)
      b_input_mask = b_input_mask.to(device)
      b_labels_r = b_labels_r.to(device)
      b_labels_c = b_labels_c.to(device)
      emotion_vectors = emotion_vectors.to(device)
      # Telling the model not to compute or store gradients,
      # saving memory and speeding up validation
      with torch.no_grad():
        # Forward pass, calculate validation loss
        loss = model(b_input_ids,emotion_vectors,attention_mask=b_input_mask,labels_r=b_labels_r,labels_c=b_labels_c)
        loss = loss.mean()
        eval_loss += loss.item()
        num_eval_samples += b_labels_c.size(0)

    epoch_eval_loss = eval_loss/num_eval_samples
    valid_loss_set.append(epoch_eval_loss)

    print("Valid loss: {}".format(epoch_eval_loss))

    if lowest_eval_loss == None:
      lowest_eval_loss = epoch_eval_loss
      # save model
      save_model(model, model_save_path, actual_epoch,\
                 lowest_eval_loss, train_loss_set, valid_loss_set)
    else:
      if epoch_eval_loss < lowest_eval_loss:
        lowest_eval_loss = epoch_eval_loss
        # save model
        save_model(model, model_save_path, actual_epoch,\
                   lowest_eval_loss, train_loss_set, valid_loss_set)
    print("\n")

  return model, train_loss_set, valid_loss_set


def save_model(model, save_path, epochs, lowest_eval_loss, train_loss_hist, valid_loss_hist):
  """
  Save the model to the path directory provided
  """
  model_to_save = model.module if hasattr(model, 'module') else model
  checkpoint = {'epochs': epochs, \
                'lowest_eval_loss': lowest_eval_loss,\
                'state_dict': model_to_save.state_dict(),\
                'train_loss_hist': train_loss_hist,\
                'valid_loss_hist': valid_loss_hist
               }
  torch.save(checkpoint, save_path)
  print("Saving model at epoch {} with validation loss of {}".format(epochs,\
                                                                     lowest_eval_loss))
  

  return
  
def load_model(save_path):
  """
  Load the model from the path directory provided
  """
  checkpoint = torch.load(save_path)
  model_state_dict = checkpoint['state_dict']
  model = ElectraForMultiLabelSequenceClassification(num_labels=model_state_dict["classifier.weight"].size()[0])
  model.load_state_dict(model_state_dict)

  epochs = checkpoint["epochs"]
  lowest_eval_loss = checkpoint["lowest_eval_loss"]
  train_loss_hist = checkpoint["train_loss_hist"]
  valid_loss_hist = checkpoint["valid_loss_hist"]
  
  return model, epochs, lowest_eval_loss, train_loss_hist, valid_loss_hist

Now we need to define for how many epochs our model will be runing and were we store the best model

In [ ]:
num_epochs=50

cwd = os.getcwd()
model_save_path = output_model_file = os.path.join(cwd, "/Seed_SharedWeights_DoubleHead_50epochs_Bert.bin")
 

mow we will just train the model

In [ ]:
torch.cuda.empty_cache()

In [ ]:
model, train_loss_set, valid_loss_set = train(model=model,\
                                              num_epochs=num_epochs,\
                                              optimizer=optimizer,\
                                              train_dataloader=train_dataloader,\
                                              valid_dataloader=validation_dataloader,\
                                              model_save_path=model_save_path,\
                                              device="cuda")
torch.cuda.empty_cache()

Now that our model has been trained we will load the best version of the model that acheived the lowes loss on the validation set during the data and use it to predict the test data

In [ ]:
import matplotlib.pyplot as plt
def plot_loss(train_loss_set, valid_loss_set):
    plt.figure(figsize=(10, 6))
    plt.plot(range(1, len(train_loss_set) + 1), train_loss_set, label='Train Loss')
    plt.plot(range(1, len(valid_loss_set) + 1), valid_loss_set, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.legend()
    plt.show()



In [ ]:
# 调用绘制函数
plot_loss(train_loss_set, valid_loss_set)

In [ ]:
checkpoint = torch.load(model_save_path)
model_state_dict = checkpoint['state_dict']
model = BertSharedWeights_Multi_Task(5,4,806,512,2)
model.load_state_dict(model_state_dict,strict=False)

In [ ]:
import matplotlib.pyplot as plt


To generate predictions from the results provided by our model for both regression and classification task we define a Predict function that gives us both predictions

In [ ]:
def Predict(model, features,emotions,mask,num_label_c,num_label_r, device="cpu", batch_size=32):
  num_iter = math.ceil(features.shape[0]/batch_size)# define the number of iteration in all the data by batches
  model.eval() #Indicate that we will use the model to evaluation purposes

  c_pred_probs = np.array([]).reshape(0, num_label_c)# Create dummpy classification predictions 'all 0' vectors that we will modify
  r_pred_probs = np.array([]).reshape(0, num_label_r)# Create dummpy regression predictions 'all 0' vectors that we will modify
  model.to(device)
  d={}
  
  for i in range(num_iter):# for each batch
    X = features[i*batch_size:(i+1)*batch_size,:]#Get the features of the passed data
    Y = emotions[i*batch_size:(i+1)*batch_size,:]#Get the emotions of the passed data
    masks = mask[i*batch_size:(i+1)*batch_size,:]#Get the masks of the passed data
    X = X.to(device)#transfer them to the device 'exmp cuda'
    Y = Y.to(device)
    masks = masks.to(device)#transfer them to the device 'exmp cuda'
    # print(X.shape)
    # print(Y.shape)
    with torch.no_grad():
      logits_c,logits_r = model(input_ids_list=X,emotion_vectors=Y,attention_mask=masks) # predict the values of the passed batch of data
      #logits = logits.sigmoid().detach().cpu().numpy()
      logits_c = logits_c.sigmoid().detach().cpu().numpy()# for the classification head we will aplly a sigmoid function as like we said the classification labels are binary labels
      # print(c_pred_probs.shape)
      # print(logits_c.shape)
      c_pred_probs = np.vstack([c_pred_probs, logits_c])# stack them to the pr-defined dummy classification variable
      logits_r = logits_r.detach().cpu().numpy()# For the regression values wi well use directly the linear layer output because the regression labels are real values
      r_pred_probs = np.vstack([r_pred_probs, logits_r])# stack them to the pr-defined dummy regression variable
  d['class_pred']=np.round(c_pred_probs) # to get binary values for the sigmoids probability values we will use the .round() function for the classification results
  d['reg_pred']=r_pred_probs
  
  return d
  


####Generate predictions for the test set

In [ ]:
pred_probs_bert = Predict(model, test_input_ids,test_data_emotion_vector,test_attention_masks, 4,5, device="cuda", batch_size=64)


In [ ]:
pred_probs_bert.keys()

In [ ]:
c_pred_probs=pred_probs_bert['class_pred']
c_pred_probs

Now we will add the predictions classification labels columns for each original label

In [ ]:
label_cols = ['introverted', 'intuitive', 'thinking', 'perceiving']

test["pred_introverted"] = c_pred_probs[:,0]
test["pred_intuitive"] = c_pred_probs[:,1]
test["pred_thinking"] = c_pred_probs[:,2]
test["pred_perceiving"] = c_pred_probs[:,3]


To take a closure look to the classification performance of our model

In [ ]:
test.drop(['author','agreeableness'	,'openness'	,'conscientiousness'	,'extraversion',	'neuroticism'],axis=1).head()

In [ ]:
real_values=test[label_cols]
predicted_values=test[['pred_introverted','pred_intuitive','pred_thinking','pred_perceiving']]

To evaluate the performance of our classification head we will use different metrics such as f1 score, accuracy, precison etc.. for each class

In [ ]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
name_columns=real_values.columns.values
metrics={}
metrics["accuracy_score"]=accuracy_score
metrics["precision_score"]=precision_score
metrics["recall_score"]=recall_score
metrics["f1_score"]=f1_score



In [ ]:
test_evaluation_values={}
for column in name_columns:
    test_evaluation_values[str(column)]=[]

In [ ]:
for k,v in  metrics.items():
  for column in name_columns:
    test_evaluation_values[str(column)].append(v(real_values[column].values,predicted_values["pred_"+str(column)].values))



In [ ]:
d=pd.DataFrame(data=test_evaluation_values,index=list(metrics.keys())).T # F1 score, recall is always better
avg=[]
avg=[d['accuracy_score'].mean(),d['precision_score'].mean(),d['recall_score'].mean(),d['f1_score'].mean()]
d=d.append(dict(zip(d.columns,avg)), ignore_index=True)
d.index=list(name_columns)+['avg']

In [ ]:
#all shaerd saved model
d

#Evaluating the regression head 

like we said our prediction method produce two different predictions. The classification prediction that we evaluated it above and the regression predictions

In [ ]:
r_pred_probs=pred_probs_bert['reg_pred']
r_pred_probs

Now we will add the predictions regression labels columns for each original label

In [ ]:
label_cols = ["agreeableness",	"openness",	"conscientiousness",	"extraversion",	"neuroticism"]

test["pred_agreeableness"] = r_pred_probs[:,0]
test["pred_openness"] = r_pred_probs[:,1]
test["pred_conscientiousness"] = r_pred_probs[:,2]
test["pred_extraversion"] = r_pred_probs[:,3]
test["pred_neuroticism"] = r_pred_probs[:,4]


In [ ]:
test.drop(['author','introverted',	'intuitive',	'thinking',	'perceiving','pred_introverted',	'pred_intuitive',	'pred_thinking',	'pred_perceiving'],axis=1).head()

In [ ]:
real_values=test[label_cols]
label_cols = ["agreeableness",	"openness",	"conscientiousness",	"extraversion",	"neuroticism"]

predicted_values=test[['pred_agreeableness','pred_openness','pred_conscientiousness','pred_extraversion','pred_neuroticism']]



```
# This is formatted as code
```

we acheivef the correlation between the predicted and the original values which overpass the value provided by the state of the art basline for the Pandora datset

In [ ]:
#all shared
audtorch.metrics.functional.pearsonr(torch.tensor(real_values.values), torch.tensor(predicted_values.values), batch_first=True).mean()


To more investigate the perfomance of our model we used different other metrics such as mean square error, r2 score, the mean absolute error etc.. 

In [ ]:
from sklearn.metrics import mean_squared_error,max_error,mean_absolute_error,mean_squared_log_error,median_absolute_error,r2_score
name_columns=real_values.columns.values
metrics={}
metrics["mean_squared_error"]=mean_squared_error
metrics["max_error"]=max_error
metrics["mean_absolute_error"]=mean_absolute_error
# metrics["mean_squared_log_error"]=mean_squared_log_error
metrics["median_absolute_error"]=median_absolute_error
metrics["r2_score"]=r2_score



In [ ]:
test_evaluation_values={}
for column in name_columns:
    test_evaluation_values[str(column)]=[]

In [ ]:
for k,v in  metrics.items():
  for column in name_columns:
    if column !='person r correlation':
      test_evaluation_values[str(column)].append(v(real_values[column].values,predicted_values["pred_"+str(column)].values))
    else:
      test_evaluation_values[str(column)].append(v(real_values[[column]].values,predicted_values[["pred_"+str(column)]].values).mean())


In [ ]:
d=pd.DataFrame(data=test_evaluation_values,index=list(metrics.keys())).T # F1 score, recall is always better
avg=[]
# avg=[d['mean_squared_error'].mean(),d['max_error'].mean(),d['mean_absolute_error'].mean(),d['mean_squared_log_error'].mean(),d['median_absolute_error'].mean(),d['r2_score'].mean()]
avg=[d['mean_squared_error'].mean(),d['max_error'].mean(),d['mean_absolute_error'].mean(),d['median_absolute_error'].mean(),d['r2_score'].mean()]
d=d.append(dict(zip(d.columns,avg)), ignore_index=True)
d.index=list(name_columns)+['avg']

In [ ]:
#all shared weights
d